In [25]:
import sys
import gym
#import pylab
import random
import numpy as np
from collections import deque
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import Sequential

from scores.score_logger import ScoreLogger

EPISODES = 300

In [26]:
# Double DQN Agent for the Cartpole
# it uses Neural Network to approximate q function
# and replay memory & target q network
class DoubleDQNAgent:
    def __init__(self, state_size, action_size):
        # if you want to see Cartpole learning, then change to True
        self.render = False
        self.load_model = False
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # these is hyper parameters for the Double DQN
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.01
        self.batch_size = 64
        self.train_start = 1000
        # create replay memory using deque
        self.memory = deque(maxlen=2000)

        # create main model and target model
        self.model = self.build_model()
        self.target_model = self.build_model()

        # initialize target model
        self.update_target_model()

        if self.load_model:
            self.model.load_weights("./save_model/cartpole_ddqn.h5")

    # approximate Q function using Neural Network
    # state is input and Q Value of each action is output of network
    def build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu',
                        kernel_initializer='he_uniform'))
        model.add(Dense(24, activation='relu',
                        kernel_initializer='he_uniform'))
        model.add(Dense(self.action_size, activation='linear',
                        kernel_initializer='he_uniform'))
        model.summary()
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    # after some time interval update the target model to be same with model
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # get action from model using epsilon-greedy policy
    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model.predict(state)
            return np.argmax(q_value[0])

    # save sample <s,a,r,s'> to the replay memory
    def append_sample(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    # pick samples randomly from replay memory (with batch_size)
    def train_model(self):
        if len(self.memory) < self.train_start:
            return
        batch_size = min(self.batch_size, len(self.memory))
        mini_batch = random.sample(self.memory, batch_size)

        update_input = np.zeros((batch_size, self.state_size))
        update_target = np.zeros((batch_size, self.state_size))
        action, reward, done = [], [], []

        for i in range(batch_size):
            update_input[i] = mini_batch[i][0]
            action.append(mini_batch[i][1])
            reward.append(mini_batch[i][2])
            update_target[i] = mini_batch[i][3]
            done.append(mini_batch[i][4])

        target = self.model.predict(update_input)
        target_next = self.model.predict(update_target)
        target_val = self.target_model.predict(update_target)

        for i in range(self.batch_size):
            # like Q Learning, get maximum Q value at s'
            # But from target model
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                # the key point of Double DQN
                # selection of action is from model
                # update is from target model
                a = np.argmax(target_next[i])
                target[i][action[i]] = reward[i] + self.discount_factor * (
                    target_val[i][a])

        # make minibatch which includes target q value and predicted q value
        # and do the model fit!
        self.model.fit(update_input, target, batch_size=self.batch_size,
                       epochs=1, verbose=0)



In [27]:
ENV_NAME = "CartPole-v1"
env = gym.make(ENV_NAME)
# get size of state and action from environment
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

score_logger = ScoreLogger(ENV_NAME)

agent = DoubleDQNAgent(state_size, action_size)

scores, episodes = [], []
run = 0

"""
Implement the noisy reward
set the noise power noise_power
"""
noise_power = 0.1
reward_space = [-1,1]

for e in range(EPISODES):
    done = False
    score = 0
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    step = 0
    run += 1
    while not done:
        step += 1
        if agent.render:
            env.render()

        # get action for the current state and go one step in environment
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        # if an action make the episode end, then gives penalty of -100
        #reward = reward if not done or score == 499 else -100
        reward = reward if not done else -reward
        
        random_number = np.random.uniform(0,1,1)
        if random_number < noise_power:
            reward = -reward

        # save the sample <s, a, r, s'> to the replay memory
        agent.append_sample(state, action, reward, next_state, done)
        # every time step do the training
        agent.train_model()
        score += reward
        state = next_state

        if done:
            # every episode update the target model to be same with model
            agent.update_target_model()

            # every episode, plot the play time
            #score = score if score == 500 else score + 100
            #scores.append(score)
            
            print("Run: " + str(run) + ", exploration: " + str(agent.epsilon) + ", score: " + str(step))
            score_logger.add_score(step, run)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 24)                120       
_________________________________________________________________
dense_38 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_39 (Dense)             (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 24)                120       
_________________________________________________________________
dense_41 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_42 (De

/Users/aashishkumar/Documents/Projects/Robust-RL/scores/score_logger.py:77: RankWarning: Polyfit may be poorly conditioned
  z = np.polyfit(np.array(trend_x), np.array(y[1:]), 1)


Scores: (min: 14, avg: 16, max: 20)

Run: 4, exploration: 0.9267809647166116, score: 28
Scores: (min: 14, avg: 19, max: 28)

Run: 5, exploration: 0.9066044494080763, score: 22
Scores: (min: 14, avg: 19.6, max: 28)

Run: 6, exploration: 0.8815592697443159, score: 28
Scores: (min: 14, avg: 21, max: 28)

Run: 7, exploration: 0.8684280445126921, score: 15
Scores: (min: 14, avg: 20.142857142857142, max: 28)

Run: 8, exploration: 0.8512234991370281, score: 20
Scores: (min: 14, avg: 20.125, max: 28)

Run: 9, exploration: 0.838544138970058, score: 15
Scores: (min: 14, avg: 19.555555555555557, max: 28)

Run: 10, exploration: 0.8268805241487632, score: 14
Scores: (min: 14, avg: 19, max: 28)

Run: 11, exploration: 0.8153791427799216, score: 14
Scores: (min: 14, avg: 18.545454545454547, max: 28)

Run: 12, exploration: 0.8088788946494789, score: 8
Scores: (min: 8, avg: 17.666666666666668, max: 28)

Run: 13, exploration: 0.7920612314455105, score: 21
Scores: (min: 8, avg: 17.923076923076923, max: 28

Run: 85, exploration: 0.31519071309757335, score: 12
Scores: (min: 8, avg: 13.576470588235294, max: 53)

Run: 86, exploration: 0.31236531710902116, score: 9
Scores: (min: 8, avg: 13.523255813953488, max: 53)

Run: 87, exploration: 0.3095652482070879, score: 9
Scores: (min: 8, avg: 13.471264367816092, max: 53)

Run: 88, exploration: 0.3058708285836198, score: 12
Scores: (min: 8, avg: 13.454545454545455, max: 53)

Run: 89, exploration: 0.3022204989748846, score: 12
Scores: (min: 8, avg: 13.438202247191011, max: 53)

Run: 90, exploration: 0.2989126458468155, score: 11
Scores: (min: 8, avg: 13.411111111111111, max: 53)

Run: 91, exploration: 0.29593693465058934, score: 10
Scores: (min: 8, avg: 13.373626373626374, max: 53)

Run: 92, exploration: 0.2929908470157829, score: 10
Scores: (min: 8, avg: 13.33695652173913, max: 53)

Run: 93, exploration: 0.2894942299343089, score: 12
Scores: (min: 8, avg: 13.32258064516129, max: 53)

Run: 94, exploration: 0.2848968999718006, score: 16
Scores: (min:

KeyboardInterrupt: 

In [28]:
agent.model.save("my_model_noisy_reward.h5")